In [1]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.3/286.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.7/713.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93

In [2]:
from skfp.datasets.moleculenet import load_bace

smiles_list, y = load_bace()

print(f"Example molecule: {smiles_list[0]}")
print(f"Example class: {y[3]}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Example molecule: O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2ccccc2C)C)CC1(C)C
Example class: 1


In [3]:
from skfp.preprocessing import MolFromSmilesTransformer

mol_from_smiles = MolFromSmilesTransformer()

mols = mol_from_smiles.transform(smiles_list)

In [4]:
import numpy as np
from skfp.model_selection import scaffold_train_test_split


train_idxs, test_idxs = scaffold_train_test_split(
    mols, test_size=0.2, return_indices=True
)

# split mols and labels
mols_train = np.array(mols)[train_idxs]
mols_test = np.array(mols)[test_idxs]

y_train = y[train_idxs]
y_test = y[test_idxs]

print(f"Train set size: {len(mols_train)}")
print(f"Test set size: {len(mols_test)}")

Train set size: 1210
Test set size: 303


In [5]:
from skfp.preprocessing import MolStandardizer


standardizer = MolStandardizer()

mols_train = standardizer.transform(mols_train)
mols_test = standardizer.transform(mols_test)

In [6]:
from skfp.fingerprints import ECFPFingerprint


# create fingerprint transformer object
ecfp_fp = ECFPFingerprint()

# transform molecules into feature vectors
X_train_ecfp = ecfp_fp.transform(mols_train)
X_test_ecfp = ecfp_fp.transform(mols_test)

print(f"Fingerprint data shape: {X_train_ecfp.shape}")
print(f"Example vector: {X_train_ecfp[0]}")

Fingerprint data shape: (1210, 2048)
Example vector: [0 1 0 ... 0 0 0]


In [8]:
# import Tanimoto count distance from scikit-fingerprints
from sklearn.neighbors import KNeighborsClassifier
from skfp.distances import tanimoto_count_distance
from skfp.metrics import multioutput_auroc_score

# create kNN model with appropriate metric
clf = KNeighborsClassifier(n_jobs=-1, metric=tanimoto_count_distance)

# fit, predict
clf.fit(X_train_ecfp, y_train)
y_pred = clf.predict_proba(X_test_ecfp)[:, 1]

# calculate and print AUROC score
print(f"ECFP AUROC for kNN: {multioutput_auroc_score(y_test, y_pred):.2%}")

ECFP AUROC for kNN: 77.95%
